# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Notes
- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs
- This notebook does not include running the CtF game with the RL policy. Using the network will be separately scripted in policy/policy_RL1.py.
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb

## TODO:

- enemy with different policies (zero, patrol)
- stochastic interaction
- Reward -> only 100 for completion (with small observation)

### Sampling
- [x] Mini-batch to update 'average' gradient
- [x] Experience Replay for Random Sampling
    - [ ] Importance Sampling
    
### Move onto Deterministic
- [ ] DDPG and MADDPG

### Stability and Reducing Variance
- [ ] Target Network
- [ ] TRPO
- [ ] PPO

### Multiprocessing
- [ ] Multiprocessing for Synchronous Training (A2C)
    - [ ] Asynchronous Training (A3C)

In [1]:
!rm -rf logs/B4R4_Rzero_AC_MonteCarlo/ model/B4R4_Rzero_AC_MonteCarlo

In [2]:
TRAIN_NAME='B4R4_Rzero_AC_MonteCarlo'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME
GPU_CAPACITY=0.2 # gpu capacity in percentage

In [3]:
import os

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random

# the modules that you can use to generate the policy.
import policy.patrol 
import policy.random
import policy.simple # custon written policy
import policy.policy_RL
import policy.zeros

# Data Processing Module
from DataModule import one_hot_encoder
from Utils import MovingAverage as MA
from Utils import Experience_buffer, discount_rewards

In [4]:
#device_lib.list_local_devices()

## Hyperparameters

In [ ]:
# Replay Variables
total_episodes = 5000000 #Set total number of episodes to train agent on.
max_ep = 150
update_frequency = 64
mini_batch = 64
batch_size = 2000
experience_size=10000

# Saving Related
save_network_frequency = 1000
save_stat_frequency = 100
moving_average_step = 100

# Training Variables
LEARNING_RATE_FIX = True
LEARNINGRATE_ACTOR  = 1e-3
LEARNINGRATE_CRITIC = 3e-3
LR_ACTOR_DECAY = 0.9995
LR_CRITIC_DECAY = 0.995
LR_ACTOR_FINAL = 1e-4
LR_CRITIC_FINAL = 5e-4
gamma = 0.9
discount_factor = 0.9

# Env Settings
MAP_SIZE = 10
VISION_RANGE = 4
VISION_dX, VISION_dY = 2*VISION_RANGE+1, 2*VISION_RANGE+1

## Environment Setting

In [ ]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [ ]:
env = gym.make("cap-v0") # initialize the environment
#plt.imshow(env.render(mode='rgb_array'))
policy_red = policy.zeros.PolicyGen(env.get_map, env.get_team_red)
action_space = 5
n_agent = len(env.get_team_blue)
print('red number : ', len(env.get_team_red))
print('blue number : ', len(env.get_team_blue))

red number :  4
blue number :  4


## Policy Network

In [ ]:
class Agent():
    def __init__(self, in_size, action_size, grad_clip_norm):
        # Parameters
        self.grad_clip_norm = grad_clip_norm
        
        # Learning Rate Variables
        self.learning_rate_actor = tf.placeholder(tf.float32, shape=None, name='learning_rate_actor')
        self.learning_rate_critic = tf.placeholder(tf.float32, shape=None, name='learning_rate_critic')
        
        # Placeholders
        with tf.name_scope('holders'):
            self.state_input = tf.placeholder(shape=in_size,dtype=tf.float32, name='state')
            self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
            self.action_OH = tf.one_hot(self.action_holder, action_size)
            self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
            self.target_value_holder = tf.placeholder(shape=[None], dtype=tf.float32, name='target')
            self.advantage_holder = tf.placeholder(shape=[None], dtype=tf.float32, name='adv')            

        
        # Feed-Forward Network
        layer = slim.conv2d(self.state_input, 32, [3,3],# activation_fn=tf.nn.relu,
                            weights_initializer=layers.xavier_initializer_conv2d(),
                            biases_initializer=tf.zeros_initializer(),
                            padding='VALID',
                            scope='conv_1')
        layer = slim.avg_pool2d(layer, [2,2])
        layer = slim.conv2d(layer, 64, [2,2],# activation_fn=tf.nn.relu,
                            weights_initializer=layers.xavier_initializer_conv2d(),
                            biases_initializer=tf.zeros_initializer(),
                            padding='VALID',
                            scope='conv_2')
        self.layer = slim.flatten(layer)
        #adv_stream, val_stream = tf.split(layer, num_or_size_splits=2)
        
        # Actor stream
        actor = layers.fully_connected(self.layer, 128,
                                    activation_fn=tf.nn.relu,
                                    scope='actor_1')
        self.actor = layers.fully_connected(actor, action_size,
                                    activation_fn=None,
                                    scope='actor_2')
        self.actor_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='conv')+tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='actor')
        print('Actor Network')
        for var in self.actor_vars:
            print(var)
        self.output = tf.nn.softmax(self.actor, name='action')

        # Value stream
        self.critic = layers.fully_connected(self.layer, 1,
                                    activation_fn=None,
                                    scope='critic_1')
        self.critic = tf.reshape(self.critic, [-1])
        self.critic_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='conv') + tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='critic')
        print('Critic Network')
        for var in self.critic_vars:
            print(var)
        
        # Feed Backward
        # - compute the  loss, and use it to find gradient, and update the network
        # - May be need to add bootstrap value at the end of the value
        with tf.variable_scope('critic_train'):
            self.loss_critic = tf.reduce_mean(tf.square(self.target_value_holder - self.critic))
            self.optimizer_critic = tf.train.AdamOptimizer(self.learning_rate_critic)
            self.grads_critic = self.optimizer_critic.compute_gradients(self.loss_critic, self.critic_vars)
            if self.grad_clip_norm:
                self.grads_critic = [(tf.clip_by_norm(grad, self.grad_clip_norm), var) for grad, var in self.grads_critic]
            self.grad_holders_critic = [(tf.Variable(var, trainable=False, dtype=tf.float32, name=var.op.name+'_buffer'), var) for var in self.critic_vars]
            self.update_critic = self.optimizer_critic.apply_gradients(self.grad_holders_critic)

        with tf.variable_scope('actor_train'):
            self.entropy = -tf.reduce_mean(self.output * tf.log(self.output), name='entropy')
            self.responsible_outputs = tf.reduce_sum(self.output * self.action_OH, [1])
            self.action_function = tf.log(self.responsible_outputs)
            self.loss_actor = -tf.reduce_sum(self.action_function*self.advantage_holder)
            self.optimizer_actor = tf.train.AdamOptimizer(self.learning_rate_actor)
            self.grads_actor = self.optimizer_actor.compute_gradients(self.loss_actor, self.actor_vars)
            if self.grad_clip_norm:
                self.grads_actor = [(tf.clip_by_norm(grad, self.grad_clip_norm), var) for grad, var in self.grads_actor]
            self.grad_holders_actor = [(tf.Variable(var, trainable=False, dtype=tf.float32, name=var.op.name+'_buffer'), var) for var in self.actor_vars]
            self.update_actor = self.optimizer_actor.apply_gradients(self.grad_holders_actor)
            #self.update_actor = self.optimizer_actor.apply_gradients(self.grads_actor)
            
        with tf.name_scope('Loss'):
            self.loss = self.loss_critic + self.loss_actor - self.entropy * 0.01
            #self.loss = self.loss_critic + self.loss_actor
            
        '''with tf.name_scope('update'):
            self.accumulate_gradient = tf.group([tf.assign_add(a[0],b[0]) for a,b in zip(self.grad_holders_actor, self.grads_actor)],
                                                [tf.assign_add(a[0],b[0]) for a,b in zip(self.grad_holders_critic, self.grads_critic)])
            self.clear_batch = tf.group([tf.assign(a[0],a[0]*0.0) for a in self.grad_holders_actor],
                                        [tf.assign(a[0],a[0]*0.0) for a in self.grad_holders_critic])
            self.update_batch = tf.group(self.update_actor, self.update_critic)'''
            
        with tf.name_scope('update'):
            # update using total loss
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate_actor)
            self.grads = self.optimizer.compute_gradients(self.loss)
            if self.grad_clip_norm:
                self.grads = [(tf.clip_by_norm(grad, self.grad_clip_norm), var) for grad, var in self.grads]
            self.grad_holders = [(tf.Variable(var, trainable=False, dtype=tf.float32, name=var.op.name+'_buffer'), var) for var in tf.trainable_variables()]
            self.update_batch = self.optimizer.apply_gradients(self.grad_holders)
            
            self.accumulate_gradient = tf.group([tf.assign_add(a[0],b[0]) for a,b in zip(self.grad_holders, self.grads)]) # add gradient to buffer
            self.clear_batch = tf.group([tf.assign(a[0],a[0]*0.0) for a in self.grad_holders])
            
        # Summary
        # Histogram output
        with tf.variable_scope('debug_parameters'):
            tf.summary.histogram('output', self.output)
            tf.summary.histogram('actor', self.actor)
            tf.summary.histogram('critic', self.critic)        
            tf.summary.histogram('action', self.action_holder)
        
        # Graph summary Loss
        with tf.variable_scope('summary'):
            tf.summary.scalar(name='actor_loss', tensor=self.loss_actor)
            tf.summary.scalar(name='critic_loss', tensor=self.loss_critic)
            tf.summary.scalar(name='total_loss', tensor=self.loss)
            tf.summary.scalar(name='Entropy', tensor=self.entropy)
        
        with tf.variable_scope('weights_bias'):
            # Histogram weights and bias
            for var in slim.get_model_variables():
                tf.summary.histogram(var.op.name, var)
                
        with tf.variable_scope('gradients'):
            # Histogram Gradients
            for var, grad in zip(self.actor_vars, self.grads_actor):
                tf.summary.histogram(var.op.name+'/grad_actor', grad[0])
            for var, grad in zip(self.critic_vars, self.grads_critic):
                tf.summary.histogram(var.op.name+'/grad_critic', grad[0])
        
        with tf.variable_scope('Learning_Rate'):
            # Learning Rate
            tf.summary.scalar(name='actor_lr', tensor=self.learning_rate_actor)
            tf.summary.scalar(name='critic_lr', tensor=self.learning_rate_critic)

In [ ]:
tf.reset_default_graph() # Clear the Tensorflow graph.
myAgent = Agent(in_size=[None,VISION_dX,VISION_dY,6],action_size=5, grad_clip_norm=20) #Load the agent.
with tf.variable_scope('global_step'):
    global_step = tf.Variable(0, trainable=False, name='global_step') # global step
    increment_global_step_op = tf.assign(global_step, global_step+1)
merged = tf.summary.merge_all()

Actor Network
<tf.Variable 'conv_1/weights:0' shape=(3, 3, 6, 32) dtype=float32_ref>
<tf.Variable 'conv_1/biases:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv_2/weights:0' shape=(2, 2, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'actor_1/weights:0' shape=(256, 128) dtype=float32_ref>
<tf.Variable 'actor_1/biases:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'actor_2/weights:0' shape=(128, 5) dtype=float32_ref>
<tf.Variable 'actor_2/biases:0' shape=(5,) dtype=float32_ref>
Critic Network
<tf.Variable 'conv_1/weights:0' shape=(3, 3, 6, 32) dtype=float32_ref>
<tf.Variable 'conv_1/biases:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv_2/weights:0' shape=(2, 2, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'critic_1/weights:0' shape=(256, 1) dtype=float32_ref>
<tf.Variable 'critic_1/biases:0' shape=(1,) dtype=float32_ref>


## Session

In [ ]:
# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_CAPACITY, allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#sess = tf.Session()

ma_reward = MA(moving_average_step)
ma_length = MA(moving_average_step)
ma_captured = MA(moving_average_step)

# Setup Save and Restore Network
saver = tf.train.Saver(tf.global_variables())
writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")

Initialized Variables


In [ ]:
def record(summary_):
    with tf.device('/cpu:0'): 
        summary = tf.Summary()
        summary.value.add(tag='Records/mean_reward', simple_value=ma_reward())
        summary.value.add(tag='Records/mean_length', simple_value=ma_length())
        summary.value.add(tag='Records/mean_succeed', simple_value=ma_captured())
        writer.add_summary(summary, sess.run(global_step))
        
        #summary_str = sess.run(merged,feed_dict={myAgent.state_input:obs})
        writer.add_summary(summary_, sess.run(global_step))
        
        writer.flush()

In [ ]:
def policy_rollout(EXPLORE=False, DETERMINISTIC=True):
    # Run single episode, return the results
    s = env.reset(map_size=MAP_SIZE, policy_red=policy_red)
    #obs = one_hot_encoder(s, env.get_team_blue) # partial observation
    obs_next = one_hot_encoder(env._env, env.get_team_blue, VISION_RANGE)
    
    ep_history = []
    indv_history = [[] for _ in range(len(env.get_team_blue))]
    was_alive = [ag.isAlive for ag in env.get_team_blue]
    
    prev_reward=0
    total_reward = 0
    frame=0
    for frame in range(max_ep+1):
        obs = obs_next
        
        if not EXPLORE:
            with tf.device('/cpu:0'):
                act_prob, v0 = sess.run([myAgent.output, myAgent.critic], feed_dict={myAgent.state_input:obs})
            if DETERMINISTIC:
                act = np.argmax(act_prob, axis=1).tolist()
            else:
                #Probabilistically pick an action given our network outputs.
                act = [np.random.choice(action_space, p=act_prob[x]/sum(act_prob[x])) for x in range(n_agent)] # divide by sum : normalize
        else:
            act = np.random.randint(5,size=n_agent).tolist()

        s,r1,d,_ = env.step(act) #Get our reward for taking an action given a bandit.

        r = r1-prev_reward
        #if r >=1 and r < 100: # ignore capturing
        #    r = 0
        if frame == max_ep and d == False:
            r = -100
            r1 = -100
        total_reward += r
        
        if env.blue_win:
            r = 100
        elif frame == max_ep and d == False:
            r = -100
            d=True
        elif d:
            r = -100
        else:
            r = 0
        
        if d:
            v1 = np.array([0.0 for _ in range(n_agent)])
        else:
            obs_next = one_hot_encoder(env._env, env.get_team_blue, VISION_RANGE) # Full Observation
            v1 = sess.run(myAgent.critic, feed_dict={myAgent.state_input:obs_next})
        
        # Push history for individual that 'was' alive previous frame
        # [state, action, reward(later:discount), value, advantage(later)]
        for idx, agent in enumerate(env.get_team_blue):
            if was_alive[idx]:
                indv_history[idx].append([obs[idx],act[idx],r,v1[idx],v0[idx]])
      
        # State Transition
        prev_reward = r1
        was_alive = [ag.isAlive for ag in env.get_team_blue]
        
        if d:
            break
    # Policy rollout for all agents are done.
    # Calculate Advantage for individual histories
    for idx, history in enumerate(indv_history):
        if len(history)==0:
            continue
        _history = np.array(history)
        _history[:,2] = discount_rewards(_history[:,2],discount_factor)
        advantages = _history[:,2] + gamma * np.array(_history[:,3]) - np.array(_history[:,4])
        #advantages = discount_rewards(advantages,discount_factor)
        _history[:,4] = advantages
        ep_history.extend(_history)
            
    if len(ep_history) > 0:        
        ep_history = np.stack(ep_history)
    
    return [frame, ep_history, r1, env.blue_win, total_reward]

## Training

In [ ]:
if __name__=='__main__':
    ep = sess.run(global_step)
    exp_buffer = Experience_buffer(experience_shape=5, buffer_size=experience_size)
    batch_history = []
    try:
        #progbar = tf.keras.utils.Progbar(total_episodes,width=5, interval=0.5)
        sess.run(myAgent.clear_batch)
        while ep < total_episodes+1:
            # Run episode
            frame, history, reward, did_won, total_reward = policy_rollout(DETERMINISTIC=False)

            # Add history
            exp_buffer.add(history)

            if len(exp_buffer) > 0:
                batch_history = exp_buffer.flush()
                feed_dict={myAgent.learning_rate_actor :LEARNINGRATE_ACTOR,
                           myAgent.learning_rate_critic:LEARNINGRATE_CRITIC,
                           myAgent.reward_holder:batch_history[:,2],
                           myAgent.action_holder:batch_history[:,1],
                           myAgent.state_input:np.stack(batch_history[:,0]),
                           myAgent.target_value_holder:batch_history[:,2],#np.array([total_reward for _ in range(len(batch_history))]),
                           myAgent.advantage_holder:batch_history[:,4]}
                with tf.device('/gpu:0'):
                    sess.run(myAgent.accumulate_gradient, feed_dict=feed_dict)
                
                if ep % mini_batch == 0 and ep != 0:
                    with tf.device('/gpu:0'):
                        sess.run(myAgent.update_batch, feed_dict=feed_dict)
                        sess.run(myAgent.clear_batch)
                        #progbar.update(ep) # update progress bar
                    print('train completed: ', ep)
                    exp_buffer.flush()

            # decay lr
            if not LEARNING_RATE_FIX:
                LEARNINGRATE_ACTOR  = max(LEARNINGRATE_ACTOR*LR_ACTOR_DECAY,LR_ACTOR_FINAL)
                LEARNINGRATE_CRITIC = max(LEARNINGRATE_CRITIC*LR_CRITIC_DECAY,LR_CRITIC_FINAL)
            
            # summarize and record
            ma_reward.append(reward)
            ma_length.append(frame)
            ma_captured.append(env.blue_win)   
            
            if ep % save_stat_frequency == 0 and ep != 0 and len(batch_history) > 0:
                summary_ = sess.run(merged, feed_dict=feed_dict)
                record(summary_)

            # save weight
            if ep % save_network_frequency == 0 and ep != 0:
                saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_step)

            # Proceed to next episode
            ep += 1
            
            sess.run(increment_global_step_op)

    except KeyboardInterrupt:
        print('\n\nManually stopped the training (KeyboardInterrupt)');
        saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_step)
        print("save: ", sess.run(global_step), 'episodes')

train completed:  64
train completed:  128
train completed:  192
train completed:  256
train completed:  320
train completed:  384
train completed:  448
train completed:  512
train completed:  576
train completed:  640
train completed:  704
train completed:  768
train completed:  832
train completed:  896
train completed:  960
train completed:  1024
train completed:  1088
train completed:  1152
train completed:  1216
train completed:  1280
train completed:  1344
train completed:  1408
train completed:  1472
train completed:  1536
train completed:  1600
train completed:  1664
train completed:  1728
train completed:  1792
train completed:  1856
train completed:  1920
train completed:  1984
train completed:  2048
train completed:  2112
train completed:  2176
train completed:  2240
train completed:  2304
train completed:  2368
train completed:  2432
train completed:  2496
train completed:  2560
train completed:  2624
train completed:  2688
train completed:  2752
train completed:  2816
trai

train completed:  22400
train completed:  22464
train completed:  22528
train completed:  22592
train completed:  22656
train completed:  22720
train completed:  22784
train completed:  22848
train completed:  22912
train completed:  22976
train completed:  23040
train completed:  23104
train completed:  23168
train completed:  23232
train completed:  23296
train completed:  23360
train completed:  23424
train completed:  23488
train completed:  23552
train completed:  23616
train completed:  23680
train completed:  23744
train completed:  23808
train completed:  23872
train completed:  23936
train completed:  24000
train completed:  24064
train completed:  24128
train completed:  24192
train completed:  24256
train completed:  24320
train completed:  24384
train completed:  24448
train completed:  24512
train completed:  24576
train completed:  24640
train completed:  24704
train completed:  24768
train completed:  24832
train completed:  24896
train completed:  24960
train completed:

train completed:  44288
train completed:  44352
train completed:  44416
train completed:  44480
train completed:  44544
train completed:  44608
train completed:  44672
train completed:  44736
train completed:  44800
train completed:  44864
train completed:  44928
train completed:  44992
train completed:  45056
train completed:  45120
train completed:  45184
train completed:  45248
train completed:  45312
train completed:  45376
train completed:  45440
train completed:  45504
train completed:  45568
train completed:  45632
train completed:  45696
train completed:  45760
train completed:  45824
train completed:  45888
train completed:  45952
train completed:  46016
train completed:  46080
train completed:  46144
train completed:  46208
train completed:  46272
train completed:  46336
train completed:  46400
train completed:  46464
train completed:  46528
train completed:  46592
train completed:  46656
train completed:  46720
train completed:  46784
train completed:  46848
train completed: